In [60]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import ellipk
import matplotlib.animation as animation
import ipywidgets as widgets

m, L, g = 1, 1, 9.81 # Mass, Length, Acceleration
theta0, v0 = np.radians(60), 0 # Initial values

approxT = 2 * np.pi * np.sqrt(L / g) # Est of period from small displacement approximation

dt = 0.001 # Time step for numerical integration of the equation of motion (s).
thetas, vs = [theta0], [v0] # Variable initialisation
old_theta = theta0
i = 0

omegas = []

while True:
    i += 1
    t = i * dt

    old_theta, old_v = thetas[-1], vs[-1]
    omega = old_v / L # Angular velocity
    omegas.append(omega*400) # To plot d(theta)/dt later on, scaling up by 400x
    new_theta = old_theta + omega * dt

    acc = -g * np.sin(old_theta) # Tangential acceleration.
    new_v = old_v + acc * dt # New tangential velocity

    if t > approxT and new_v * old_v < 0:
        # End of period
        break

    thetas.append(new_theta) # Sequence of changing angles
    vs.append(new_v) # Sequence of changing velocities

def get_coords(angle):
    return L * np.sin(angle), -L * np.cos(angle)

def animate(theta):
    x, y = get_coords(thetas[theta])
    fig, (ax1, ax2) = plt.subplots(2)
    fig.set_size_inches(18.5, 10.5, forward=True)

    ax1.set_aspect('equal', adjustable='box')
    ax1.plot([0, x], [0, y], lw=3, c='k')
    ax1.add_patch(plt.Circle((x, y), 0.08, color='black', zorder=3))
    ax1.add_patch(plt.Circle((0, 0), L, linestyle='dashed', facecolor='none', edgecolor='grey'))
    ax1.set_xlim(-L*1.2, L*1.2)
    ax1.set_ylim(-L*1.2, L*1.2)
    ax1.xaxis.set_visible(False)
    ax1.yaxis.set_visible(False)

    ax2.set_aspect(0.5)
    xs = np.arange(0, len(omegas), 1)
    ax2.plot(xs, omegas, color='orange')
    ax2.plot(theta, omegas[theta], 'ro', markersize=10)
    ax2.spines['bottom'].set_position('center')
    ax2.set_yticklabels([]) # Remove numbers from axis
    ax2.set_xticklabels([])

    return plt.show()


widgets.interact(animate, theta = widgets.Play(min=0, max=2154, step=2, value=1, interval=0.1))


interactive(children=(Play(value=1, description='theta', interval=0, max=2154, step=2), Output()), _dom_classe…

<function __main__.animate(theta)>